### Clickhouse

In [ ]:
!pip install clickhouse-driver

#### 1 способ- через Client

In [1]:
from clickhouse_driver import Client

In [26]:
# Подключение к базе данных ClickHouse с использованием пароля
client = Client(
    host='localhost',  # Адрес сервера (можно заменить на IP, если это контейнер Docker)
    port=19000,         # Порт, на котором работает ClickHouse
    user='default',  # Имя пользователя
    password='zOvU71y+',  # Пароль
    database='default'  # База данных (необязательно, если нужно подключение к конкретной БД)
)

In [27]:
# Пример запроса
result = client.execute('SELECT * FROM log1 LIMIT 5')

In [30]:
print(result, type(result))

[(datetime.datetime(2023, 1, 1, 0, 17, 42), 14822367), (datetime.datetime(2023, 1, 1, 0, 37, 2), 14822519), (datetime.datetime(2023, 1, 1, 1, 0, 20), 14822607), (datetime.datetime(2023, 1, 1, 1, 26, 38), 14822767), (datetime.datetime(2023, 1, 1, 1, 27, 5), 14822767)] <class 'list'>


In [29]:
for row in result:
    print(row)

(datetime.datetime(2023, 1, 1, 0, 17, 42), 14822367)
(datetime.datetime(2023, 1, 1, 0, 37, 2), 14822519)
(datetime.datetime(2023, 1, 1, 1, 0, 20), 14822607)
(datetime.datetime(2023, 1, 1, 1, 26, 38), 14822767)
(datetime.datetime(2023, 1, 1, 1, 27, 5), 14822767)


In [2]:
import pandas as pd

In [29]:
df1 = pd.DataFrame(result, columns=['event_time', 'uid'])
df1

,event_time,uid
0,2023-01-01 00:17:42,14822367
1,2023-01-01 00:37:02,14822519
2,2023-01-01 01:00:20,14822607
3,2023-01-01 01:26:38,14822767
4,2023-01-01 01:27:05,14822767


#### 2 способ- через Pandahouse

In [10]:
import pandahouse as ph

In [23]:
connection = dict(
    host='http://localhost:18123',  # Адрес сервера (можно заменить на IP, если это контейнер Docker)         # Порт, на котором работает ClickHouse
    user='default',  # Имя пользователя
    password='zOvU71y+',  # Пароль
    database='default'  # База данных (необязательно, если нужно подключение к конкретной БД)
)

In [31]:
query = """
SELECT *
FROM finance
LIMIT 5
"""
df2 = ph.read_clickhouse(query, connection=connection)
df2

,event_time,revenue_usd,uid,is_test
0,2023-01-01 00:17:42,0.99,14822367,0
1,2023-01-01 00:37:02,6.99,14822519,0
2,2023-01-01 01:00:20,4.99,14822607,0
3,2023-01-01 01:26:38,4.99,14822767,0
4,2023-01-01 01:27:05,19.99,14822767,0


### Читаем данные из csv в pandas и записываем в Сlickhouse

In [ ]:
query_create = """
CREATE TABLE default.medium (
    published String,
    domain String,
    url String
)
ENGINE = Log
"""
df_create = ph.read_clickhouse(query_create, connection=connection)
df_create

In [42]:
df_in = pd.read_csv("medium.csv", sep='\t')

In [43]:
ph.to_clickhouse(df_in, 'medium', index=False, connection=connection)

92191

### PostgreSQL

In [ ]:
!pip install psycopg2 #НУЖНО ДЛЯ sqlalchemy

In [33]:
from sqlalchemy import create_engine

In [34]:
db_config = {'user': 'student', # имя пользователя
            'pwd': 'qweasd963', # пароль
            'host': '95.163.241.236',
            'port': 5432, # порт подключения
            'db': 'nordwind'} # название базы данных

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                db_config['pwd'],
                                                db_config['host'],
                                                db_config['port'],
                                                db_config['db'])

engine = create_engine(connection_string)

Коннекция к базе хранится в переменной engine. 
Теперь пишем и выполняем sql-запрос к базе данных  dash_visits, чтобы вытащить оттуда наши данные:

In [ ]:
query2 = ''' SELECT *
             FROM products
             LIMIT 5
        '''
    
df3= pd.io.sql.read_sql(query2, con = engine)

In [38]:
df3.head(5)

,product_id,product_name,supplier_id,category_id,quantity_per_unit,unit_price,units_in_stock,units_on_order,reorder_level,discontinued
0,1,Chai,8,1,10 boxes x 30 bags,18.00,39,0,10,1
1,2,Chang,1,1,24 - 12 oz bottles,19.00,17,40,25,1
2,3,Aniseed Syrup,1,2,12 - 550 ml bottles,10.00,13,70,25,0
3,4,Chef Anton's Cajun Seasoning,2,2,48 - 6 oz jars,22.00,53,0,0,0
4,5,Chef Anton's Gumbo Mix,2,2,36 boxes,21.35,0,0,0,1


### Сохраняем наши данные в файл data.csv.

In [ ]:
df.to_csv('data.csv', index=False)

In [ ]:
dash_visits = pd.read_csv('data.csv')
dash_visits.head()

In [3]:
import requests

# replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
url = 'https://www.cbr-xml-daily.ru/daily_json.js'
r2 = requests.get(url)
data = r2.json()

#print(data)
print(data.keys())
data

dict_keys(['Date', 'PreviousDate', 'PreviousURL', 'Timestamp', 'Valute'])


{'Date': '2025-01-16T11:30:00+03:00',
 'PreviousDate': '2025-01-15T11:30:00+03:00',
 'PreviousURL': '//www.cbr-xml-daily.ru/archive/2025/01/15/daily_json.js',
 'Timestamp': '2025-01-15T18:00:00+03:00',
 'Valute': {'AUD': {'ID': 'R01010',
   'NumCode': '036',
   'CharCode': 'AUD',
   'Nominal': 1,
   'Name': 'Австралийский доллар',
   'Value': 63.6894,
   'Previous': 64.0178},
  'AZN': {'ID': 'R01020A',
   'NumCode': '944',
   'CharCode': 'AZN',
   'Nominal': 1,
   'Name': 'Азербайджанский манат',
   'Value': 60.4752,
   'Previous': 60.8459},
  'GBP': {'ID': 'R01035',
   'NumCode': '826',
   'CharCode': 'GBP',
   'Nominal': 1,
   'Name': 'Фунт стерлингов',
   'Value': 125.4358,
   'Previous': 125.6979},
  'AMD': {'ID': 'R01060',
   'NumCode': '051',
   'CharCode': 'AMD',
   'Nominal': 100,
   'Name': 'Армянских драмов',
   'Value': 25.8253,
   'Previous': 26.0038},
  'BYN': {'ID': 'R01090B',
   'NumCode': '933',
   'CharCode': 'BYN',
   'Nominal': 1,
   'Name': 'Белорусский рубль',
   '

In [8]:
df5=pd.DataFrame(data)
df5

,Date,PreviousDate,PreviousURL,Timestamp,Valute
AUD,2025-01-16T11:30:00+03:00,2025-01-15T11:30:00+03:00,//www.cbr-xml-daily.ru/archive/2025/01/15/dail...,2025-01-15T18:00:00+03:00,"{'ID': 'R01010', 'NumCode': '036', 'CharCode':..."
AZN,2025-01-16T11:30:00+03:00,2025-01-15T11:30:00+03:00,//www.cbr-xml-daily.ru/archive/2025/01/15/dail...,2025-01-15T18:00:00+03:00,"{'ID': 'R01020A', 'NumCode': '944', 'CharCode'..."
GBP,2025-01-16T11:30:00+03:00,2025-01-15T11:30:00+03:00,//www.cbr-xml-daily.ru/archive/2025/01/15/dail...,2025-01-15T18:00:00+03:00,"{'ID': 'R01035', 'NumCode': '826', 'CharCode':..."
AMD,2025-01-16T11:30:00+03:00,2025-01-15T11:30:00+03:00,//www.cbr-xml-daily.ru/archive/2025/01/15/dail...,2025-01-15T18:00:00+03:00,"{'ID': 'R01060', 'NumCode': '051', 'CharCode':..."
BYN,2025-01-16T11:30:00+03:00,2025-01-15T11:30:00+03:00,//www.cbr-xml-daily.ru/archive/2025/01/15/dail...,2025-01-15T18:00:00+03:00,"{'ID': 'R01090B', 'NumCode': '933', 'CharCode'..."
BGN,2025-01-16T11:30:00+03:00,2025-01-15T11:30:00+03:00,//www.cbr-xml-daily.ru/archive/2025/01/15/dail...,2025-01-15T18:00:00+03:00,"{'ID': 'R01100', 'NumCode': '975', 'CharCode':..."
BRL,2025-01-16T11:30:00+03:00,2025-01-15T11:30:00+03:00,//www.cbr-xml-daily.ru/archive/2025/01/15/dail...,2025-01-15T18:00:00+03:00,"{'ID': 'R01115', 'NumCode': '986', 'CharCode':..."
HUF,2025-01-16T11:30:00+03:00,2025-01-15T11:30:00+03:00,//www.cbr-xml-daily.ru/archive/2025/01/15/dail...,2025-01-15T18:00:00+03:00,"{'ID': 'R01135', 'NumCode': '348', 'CharCode':..."
VND,2025-01-16T11:30:00+03:00,2025-01-15T11:30:00+03:00,//www.cbr-xml-daily.ru/archive/2025/01/15/dail...,2025-01-15T18:00:00+03:00,"{'ID': 'R01150', 'NumCode': '704', 'CharCode':..."
HKD,2025-01-16T11:30:00+03:00,2025-01-15T11:30:00+03:00,//www.cbr-xml-daily.ru/archive/2025/01/15/dail...,2025-01-15T18:00:00+03:00,"{'ID': 'R01200', 'NumCode': '344', 'CharCode':..."


455a82677f82   clickhouse              "/entrypoint.sh"         7 days ago      Up 3 hours                9009/tcp, 0.0.0.0:18123->8123/tcp, 0.0.0.0:19000->9000/tcp   some-clickhouse-server